<a href="https://colab.research.google.com/github/mloyorev/Theory/blob/main/7_OptimalSavings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install quantecon

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.8/214.8 kB 2.1 MB/s eta 0:00:00


In [3]:
from collections import namedtuple
import numpy as np
import quantecon as qe
from numba import njit, prange, int32
import matplotlib.pyplot as plt

Once **again** we use the functions:


*   argmax
*   succesive_approx



In [4]:
@njit
def argmax(list_object):
    max_val = -np.inf
    argmax_index = None
    for i, x in enumerate(list_object):
        if x > max_val:
            max_val = x
            argmax_index = i
    return argmax_index

In [5]:
def successive_approx(T,
                      x_0,
                      tolerance=1e-6,
                      max_iter=10_000,
                      print_step=25,
                      verbose=False):
    x = x_0
    error = tolerance + 1
    k = 1
    while error > tolerance and k <= max_iter:
        x_new = T(x)
        error = np.max(np.abs(x_new - x))
        if verbose and k % print_step == 0:
            print(f"Completed iteration {k} with error {error}.")
        x = x_new
        k += 1
    if error > tolerance:
        print(f"Warning: Iteration hit upper bound {max_iter}.")
    elif verbose:
        print(f"Terminated successfully in {k} iterations.")
    return x


# **Optimal savings model**

We assume that **wealth $W_{t}$ evolves** according to

$$C_{t}+W_{t+1}≤RW_{t}+Y_{t}$$

where $R$ is the gross interest rate, $Y_{t}$ is the labor income and:


*   $(W_{t})$ takes values in a finite set $W$ contained in the set $\mathbb{R}_{+}$
*   $(Y_{t})$ is a **$Q$-Markov chain** on a finite and continuous set $Y$.
*   $C_{t}\geq 0$ for all $t$.

The individual **aims to maximize the present value** of his utility function. Additionally, we suppose that the individual perceive utility from consumption. This means that the consumer's maximization problem is

$$V(w_{t},y_{t})=\max_{\left\{w_{t+\tau+1}\right\}}E_{t}\sum_{\tau=0}^{\infty}\beta^{\tau}u(c_{t})=\max_{\left\{w_{t+\tau+1}\right\}}E_{t}\sum_{\tau=0}^{\infty}\beta^{\tau}u(Rw_{t}+y_{t}-w_{t+\tau+1})$$

Since $E_{t}u(Rw_{t}+y_{t}-w_{t+1})=u(Rw_{t}+y_{t}-w_{t+1})$, this equation can be also expressed as

$$V(w_{t},y_{t})=\max_{w_{t+1}}u(Rw_{t}+y_{t}-w_{t+1})+\max_{\left\{w_{t+\tau+1}\right\}}E_{t}\sum_{\tau=1}^{\infty}\beta^{\tau}u(Rw_{t+\tau}+y_{t+\tau}-w_{t+\tau+1})$$

By making the substitution $\tau=k+1$, we obtain that

$$V(w_{t},y_{t})=\max_{w_{t+1}}u(Rw_{t}+y_{t}-w_{t+1})+\max_{\left\{w_{t+k+2}\right\}}E_{t}\sum_{k=0}^{\infty}\beta^{k+1}u(Rw_{t+k+1}+y_{t+k+1}-w_{t+k+2})$$

$$V(w_{t},y_{t})=\max_{w_{t+1}}u(Rw_{t}+y_{t}-w_{t+1})+\max_{\left\{w_{t+k+2}\right\}}\beta E_{t}\sum_{k=0}^{\infty}\beta^{k}u(Rw_{t+k+1}+y_{t+k+1}-w_{t+k+2})$$

According to the **law of iterated expectations**$^{1}$ this expression can be rewritten as

$$V(w_{t},y_{t})=\max_{w_{t+1}}u(Rw_{t}+y_{t}-w_{t+1})+\max_{\left\{w_{t+k+2}\right\}}\beta ∫_{y_{t+1}}E_{t+1}\left[\sum_{k=0}^{\infty}\beta^{k}u(Rw_{t+k+1}+y_{t+k+1}-w_{t+k+2})\right]f(y_{t+1}|y_{t})dy_{t+1}$$

$$V(w_{t},y_{t})=\max_{w_{t+1}}u(Rw_{t}+y_{t}-w_{t+1})+\beta ∫_{y_{t+1}}\max_{\left\{w_{t+k+2}\right\}}E_{t+1}\left[\sum_{k=0}^{\infty}\beta^{k}u(Rw_{t+k+1}+y_{t+k+1}-w_{t+k+2})\right]f(y_{t+1}|y_{t})dy_{t+1}$$

$$V(w_{t},y_{t})=\max_{w_{t+1}}u(Rw_{t}+y_{t}-w_{t+1})+\beta ∫_{y_{t+1}}V(w_{t+1},y_{t+1})f(y_{t+1}|y_{t})dy_{t+1}$$

Finally, **by definition**, we have that

$$V(w,y)=\max_{w'}u(Rw+y-w')+\beta E_{t}V(w',y')$$

which represents the **Bellman equation** for the stochastic dynamic programin problem of the optimal investment problem.

Once we define the model we introduce the functions needed to define the default value of the model parameter.

Additional to this functions, we suppose that the set $Y$ is discrete, such that the Bellman equation can be written as

$$V(w,y)=\max_{w'}u(Rw+y-w')+\beta \sum_{y'\in Y}V(w',y')Q(y,y')$$

where $Q(y,y')$ represents the transition matrix, and that the utility function can be described as

$$u(c)=\frac{c^{1+\gamma}}{1+\gamma}$$

In [7]:
Model = namedtuple('Model', ('beta',    # Discount factor
                             'R',       # Gross interest rate
                             'gamma',   # CRRA parameter
                             'w_grid',  # Wealth grid
                             'y_grid',  # Labor income grid
                             'Q'))      # Labor income transition matrix

In [8]:
def create_consumption_model(R=1.01,         # Gross interest rate
                             beta=0.98,      # Discount factor
                             gamma=2.5,      # CRRA parameter
                             w_min=0.01,     # Min wealth
                             w_max=5.0,      # Max wealth
                             w_size=150,     # Size of the wealth grid
                             rho=0.9, nu=0.1,   # Income parameters
                             y_size=100):    # Size of the income grid

    w_grid = np.linspace(w_min, w_max, w_size)  # Create the wealth grid
    mc = qe.tauchen(rho, nu, n=y_size)          # 'qe.tauchen' computes a Markov chain associated with a discretized version of the linear Gaussian AR(1) process
                                                #    - rho is the autocorrelation coefficient
                                                #    - nu is the standard deviation of the random process
    y_grid, Q = np.exp(mc.state_values), mc.P   # - 'y_grid' is the grid of transformed income values
                                                # - 'Q' is the transition matrix describing transition probabilities between discrete income values ​​in the stochastic process

    return Model(beta=beta, R=R, gamma=gamma, w_grid=w_grid, y_grid=y_grid, Q=Q)

Here's the RHS of the unmaximized Bellman equation

In [10]:
@njit
def B(i, j, ip, v, model):

    β, R, γ, w_grid, y_grid, Q = model          # Unpacked model parameters
    w, y, wp = w_grid[i], y_grid[j], w_grid[ip] # We use:
                                                #  - index i for access the value of current wealth within the wealth grid
                                                #  - index j for access the value of income within the income grid
                                                #  - index ip foe acces the value of next period wealth within the wealth grid
    c = R * w + y - wp
    if c > 0:
        return c**(1 - γ) / (1 - γ) + β * np.dot(v[ip, :], Q[j, :])
    return - np.inf

In order to solve this model, we have to prove that it satisfies the **Blackwell sufficiency conditions**.

### **1. Monotonicity**
Let $V,W$ two different functions that satisfy that $V(w,y)≤W(w,y)
$ $∀w,y$. If we name $w',y'$ the wealth and the labor income shock of the next period then...

$$V(w',y')≤W(w',y'),  ∀w',y'$$

If we multiply both sides of the inequality by the transition probability matrix, then...

$$V(w',y')Q(y,y')≤W(w',y')Q(y,y'),  ∀w',y'$$

The inequality also holds for the expected value of value functions, such that...

$$\sum_{y'}V(w',y')Q(y,y')≤\sum_{y'}W(w',y')Q(y,y')$$
$$β\sum_{y'}V(w',y')Q(y,y')≤β\sum_{y'}W(w',y')Q(y,y')$$

Finally we add both sides the maximization of the current utility given a particular level of current wealth and income shock, such that...

$$\max_{w'}u(w,y,w')+β\sum_{y'}V(w',y')Q(y,y')≤\max_{w'}u(w,y,w')+β\sum_{y'}W(w',y')Q(y,y')$$

Therefore, $T(V(w,y))≤T(W(w,y)), ∀w,y$, which means that the transformation **satisifes monotonicity**.

### **2. Discounting**
Let's consider a value function $V$ and a positive constant $a$, such that...

$$T(V(w,y)+a)=\max_{w'}u(w,y,w')+β∑_{y'}(V(w',y')+a)Q(y,y')$$



$$T(V(w,y)+a)=\max_{w'}u(w,y,w')+β∑_{y'}(V(w',y'))Q(y,y')+β∑_{y'}aQ(y,y')$$



$$T(V(w,y)+a)=\max_{w'}u(w,y,w')+β∑_{y'}(V(w',y'))Q(y,y')+aβ∑_{y'}Q(y,y')$$

it is fulfilled that $∑_{y'}Q(y,y')=1$, such that...

$$T(V(w,y)+a)=\max_{w'}u(w,y,w')+β∑_{y'}(V(w',y'))Q(y,y')+aβ$$



$$T(V(w,y)+a)=T(V(w,y))+aβ$$

Therefore, $T(V(w,y)+a)≤T(V(w,y))+aβ$, which means that the transformation satisfies discounting.


Since we already know that the model satisfies the Blackwell sufficiency conditions, then we are now sure that **there is a solution and that it is unique**.

Unlike previous notebooks, we will now solve the model with **three different algorithms**:


*   Value Function Iteration (VFI).
*   Howard Policy Iteration (HPI).
*   Optimistic Policy Iteration (OPI).



In order to be able to use this algorithms, we have to define the policy operator $T_{\sigma}$, the Bellman operator $T$ and the $v-$greedy policy given $v$.

In [13]:
# POLICY OPERATOR

@njit(parallel=True)          # It is compiled with numba and the inner loop can be parallelized with multiple CPU cores
def T_sigma(v, sigma, model):

    w_size, y_size = len(model.w_grid), len(model.y_grid) # Obtain the size of the wealth and income grids
    v_new = np.empty_like(v)                              # Initialize a value fuction of the size of the 'v' array

    for i in prange(w_size):                              # Iterates through the i indices of the wealth grid
                                                          # 'prange(w_size)' indicates that the loop can be paralelized
        for j in range(y_size):                           # Iterates through the j indices of the income grid
            v_new[i, j] = B(i, j, sigma[i, j], v, model)  # It gets the value of each possible combination of wealth and income
                                                          # Notiche that:
                                                          #   - i is the index of current wealth
                                                          #   - j is the index of the current income
                                                          #   - 'sigma[i, j]' is the next period wealth given the indices of current wealth and income
    return v_new

In [14]:
# BELLMAN OPERATOR

@njit(parallel=True)
def T(v, model):

    β, R, γ, w_grid, y_grid, Q = model
    w_size, y_size = len(w_grid), len(y_grid)
    v_new = np.empty_like(v)

    for i in prange(w_size):
        for j in range(y_size):
            v_new[i, j] = max([B(i, j, ip, v, model) for ip in range(w_size)])  # It gets the value of each possible combination of wealth and income
                                                                                # Notiche that:
                                                                                #   - i is the index of current wealth
                                                                                #   - j is the index of the current income
                                                                                #   - ip is the next period wealth
    return v_new

In [16]:
# V-GREEDY POLICY GIVEN V

@njit(parallel=True)
def get_greedy(v, model):

    β, R, γ, w_grid, y_grid, Q = model
    w_size, y_size = len(w_grid), len(y_grid)
    σ = np.empty_like(v, dtype=int32)

    for i in prange(w_size):
        for j in range(y_size):
            σ[i, j] = argmax([B(i, j, ip, v, model) for ip in range(w_size)]) #   For each pair (i,j), se obtain the index ip that maximize
                                                                              # the Bellman equation.
    return σ

The function below computes the value $v_{\sigma}$ of following a policy $\sigma$.

In order to compute the value of a policy, we have to solve the linear system

$$v(w,y)=u(Rw+y-\sigma(w,y))+β\sum_{y'}v(\sigma(w,y),y')Q(y',y)$$

for $v$.

Therefore, it turns out helpul to rewrite this system as

$$v(w,y)=r(w,y,\sigma(w,y))+β\sum_{w',y'}v(w',y')P_{\sigma}(w,y,w',y')$$

where $P_{\sigma}(w,y,w',y')=1\left\{w'=\sigma(w,y)\right\}Q(y',y)$

Thus the linear system can be simplied as

$$v=r_{\sigma}+P_{\sigma}Q(y,y')$$

and the solve for $v$.

Notice that:

*   $v$ is a $2$ index array.
*   $P_{\sigma}$ is a $4$ index array.

Then, the code below:

1.   Reshapes $v$ and $r_{\sigma}$ to 1D arrays and $P_{\sigma}$ to a matrix.
2.   Solves the linear system.
3.   Converts back to multi-index arrays.





In [17]:
# GET THE VALUE V_SIGMA OF POLICY SIGMA

@njit(parallel=True)
def get_value(sigma, model):

    # Unpack and set up
    beta, R, gamma, w_grid, y_grid, Q = model
    nw, ny = len(w_grid), len(y_grid)
    n = nw * ny

    # Allocate and create multi-index versions of P_σ and r_σ
    P_sigma = np.zeros((nw, ny, nw, ny))
    r_sigma = np.zeros((nw, ny))
    for i in range(nw):
        for j in range(ny):
            w, y, wp = w_grid[i], y_grid[j], w_grid[sigma[i, j]]
            c = R * w + y - wp
            r_sigma[i, j] = c**(1 - gamma) / (1 - gamma)
            for ip in range(nw):
                for jp in range(ny):
                    if ip == sigma[i, j]:
                        P_sigma[i, j, ip, jp] = Q[j, jp]

    # Reshape to standard matrix algebra form
    r_sigma = r_sigma.reshape((n,))
    P_sigma = P_sigma.reshape((n, n))

    # Solve for the value of σ
    I = np.identity(n)
    v_σ = np.linalg.solve(I - beta * P_sigma, r_sigma)

    # Return as multi-index array
    return np.reshape(v_σ, (nw, ny))